### Import Library

In [19]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from collections import Counter
from scipy.stats import  ks_2samp, chi2_contingency
import matplotlib.pyplot as plt
from scipy import stats
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Dense, Dropout

import nest_asyncio
nest_asyncio.apply()

# تنظیم بذر تصادفی برای تکرارپذیری
np.random.seed(42)
tf.random.set_seed(42)

# تنظیم زمینه اجرایی محلی
tff.backends.native.set_local_execution_context()

### Create Model 

In [2]:
# تابع ساخت مدل MLP
def create_mnist_mlp_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))  # ورودی به شکل مسطح شده (28x28 = 784)
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # لایه خروجی با 10 کلاس (softmax)
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # کامپایل کردن مدل

    return model

### Load and Split

In [3]:
# لود کردن داده‌های MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# نرمال‌سازی داده‌ها
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# مسطح کردن تصاویر
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# کدگذاری لیبل‌ها
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# تقسیم داده‌ها بین 10 کلاینت
num_clients = 10
client_data = np.array_split(x_train, num_clients)
client_labels = np.array_split(y_train, num_clients)

### Bug Injection

In [4]:
# لیستی از کلاینت‌هایی که باید دوبار نرمال‌سازی شوند
clients_to_double_normalize = [2]  # کلاینت‌های 3 (با اندیس صفر شروع می‌شود)

# دوبار نرمال‌سازی کلاینت‌های مشخص‌شده
for client_index in clients_to_double_normalize:
    client_data[client_index] = client_data[client_index] / 255.0

# پوشه‌ای برای ذخیره پارامترهای کلاینت‌ها ایجاد می‌کنیم
if not os.path.exists('client_params'):
    os.makedirs('client_params')

### Models training & save

In [12]:
# آموزش مدل‌ها برای هر کلاینت و ذخیره پارامترها
client_models = [] #لیست برای ذخیره مدل‌های هر کلاینت

base_model = create_mnist_mlp_model()  # مدل پایه

#base_model.save_weights('base_model_weights.h5') # ذخیره وزن‌های مدل پایه

for i in range(num_clients):
    print(f"Training model for client {i+1}")    
    model=create_mnist_mlp_model()
    model.set_weights(base_model.get_weights())
    
    # بررسی یکسان بودن وزن‌ها قبل از آموزش
    m1 = model.get_weights()
    b1 = base_model.get_weights()
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # کامپایل کردن مدل
    for index, (base_w, clone_w) in enumerate(zip(m1, b1)):
        if not np.array_equal(base_w, clone_w):
            print(f"Weights at index {index} are different.")
        else:
            print(f"Weights at index {index} are identical.")
    model.fit(client_data[i], client_labels[i], epochs=5, batch_size=32, verbose=1)
    #model.save_weights(f'client_params/client_{i+1}_weights.h5') # ذخیره وزن‌های هر کلاینت
    #print("**********",model.predict(x_test[:100].argmax(axis=1)))
    client_models.append(model)  # افزودن مدل به لیست
    print("Training completed for all clients.")

Training model for client 1
Weights at index 0 are identical.
Weights at index 1 are identical.
Weights at index 2 are identical.
Weights at index 3 are identical.
Weights at index 4 are identical.
Weights at index 5 are identical.
Epoch 1/5
188/188 [==============================] - 1s 6ms/step - loss: 0.6048 - accuracy: 0.8125
Epoch 2/5
188/188 [==============================] - 1s 6ms/step - loss: 0.2605 - accuracy: 0.9193
Epoch 3/5
188/188 [==============================] - 1s 6ms/step - loss: 0.1722 - accuracy: 0.9473
Epoch 4/5
188/188 [==============================] - 1s 6ms/step - loss: 0.1305 - accuracy: 0.9588
Epoch 5/5
188/188 [==============================] - 1s 8ms/step - loss: 0.1093 - accuracy: 0.9640
Training completed for all clients.
Training model for client 2
Weights at index 0 are identical.
Weights at index 1 are identical.
Weights at index 2 are identical.
Weights at index 3 are identical.
Weights at index 4 are identical.
Weights at index 5 are identical.
Epoch

### Class

In [13]:
def delta_class(models, x_test):
    num_models = len(models)
    argmax_preds = [model.predict(x_test).argmax(axis=1) for model in models]   # پیش‌بینی‌ها برای هر مدل
    diffs_matrix = np.zeros((num_models, num_models), dtype=int)    # ماتریس مربعی برای ذخیره تفاوت‌ها
    for i in range(num_models):       # پر کردن ماتریس با تفاوت پیش‌بینی‌ها
        for j in range(i+1, num_models):
            diffs = np.sum(argmax_preds[i] != argmax_preds[j])  # تعداد تفاوت‌های پیش‌بینی بین مدل i و مدل j
            diffs_matrix[i, j] = diffs
            diffs_matrix[j, i] = diffs  # ماتریس متقارن است
    return diffs_matrix

# اندازه‌گیری زمان
start_time = time.time()
diff_class = delta_class(client_models, x_test)
print("Difference Matrix for Delta Class:")
print(diff_class)

end_time = time.time()
execution_time = end_time - start_time
print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Difference Matrix for Delta Class:
[[   0  554 1743  526  523  524  511  522  582  502]
 [ 554    0 1693  545  539  578  596  576  550  571]
 [1743 1693    0 1684 1676 1741 1745 1628 1753 1727]
 [ 526  545 1684    0  525  501  496  525  575  499]
 [ 523  539 1676  525    0  531  564  504  577  501]
 [ 524  578 1741  501  531    0  527  524  636  496]
 [ 511  596 1745  496  564  527    0  526  665  509]
 [ 522  576 1628  525  504  524  526    0  633  542]
 [ 582  550 1753  575  577  636  665  633    0  608]
 [ 502  571 1727  499  501  496  509  542  608    0]]

Execution time for delta_class: 7.0223 seconds


### Score

In [26]:
def delta_score(models, x_test, threshold):
    num_models = len(models)
    diffs_matrix = np.zeros((num_models, num_models))

    # پیش‌بینی خروجی‌های هر مدل
    predictions = [model.predict(x_test) for model in models]
    
    # محاسبه‌ی argmax برای هر مدل
    argmax_preds = [np.argmax(pred, axis=1) for pred in predictions]

    # مقایسه مدل‌ها دو به دو
    for i in range(num_models):
        for j in range(i + 1, num_models):
            # مقایسه‌ی argmax ها
            argmax_diff = argmax_preds[i] != argmax_preds[j]
            diff_count = np.sum(argmax_diff)  # شمارش اختلاف در argmax
            
            # مقایسه‌ی احتمالات اگر argmax یکسان باشد
            same_argmax_indices = np.where(argmax_preds[i] == argmax_preds[j])[0]
            
            for idx in same_argmax_indices:
                # مقایسه‌ی احتمالات در همان کلاس‌های argmax
                prob_i = predictions[i][idx][argmax_preds[i][idx]]
                prob_j = predictions[j][idx][argmax_preds[j][idx]]
                
                # محاسبه اختلاف بین پیش‌بینی‌ها
                prob_diff = abs(prob_i - prob_j)
                
                # بررسی اختلاف با آستانه threshold
                if prob_diff >= threshold:
                    diff_count += 1  # شمارش اختلاف در پیش‌بینی‌ها

            # ذخیره اختلافات در هر دو موقعیت (i, j) و (j, i) برای متقارن بودن
            diffs_matrix[i, j] = diff_count
            diffs_matrix[j, i] = diff_count

    return diffs_matrix


# اجرای تابع با تعیین آستانه threshold
diff_score = delta_score(client_models, x_test, threshold=0.1)
print("Distance Matrix for Delta Score:")
print(diff_score)

start_time = time.time()
end_time = time.time()
execution_time = end_time - start_time
print(f"\nExecution time for delta_score: {execution_time:.4f} seconds")


Distance Matrix for Delta Score:
[[   0. 1450. 2310. 1348. 1394. 1330. 1363. 1396. 1460. 1353.]
 [1450.    0. 2228. 1398. 1391. 1439. 1457. 1476. 1439. 1432.]
 [2310. 2228.    0. 2174. 2200. 2190. 2215. 2128. 2322. 2240.]
 [1348. 1398. 2174.    0. 1360. 1277. 1295. 1357. 1454. 1340.]
 [1394. 1391. 2200. 1360.    0. 1319. 1352. 1332. 1488. 1329.]
 [1330. 1439. 2190. 1277. 1319.    0. 1329. 1377. 1518. 1357.]
 [1363. 1457. 2215. 1295. 1352. 1329.    0. 1330. 1548. 1334.]
 [1396. 1476. 2128. 1357. 1332. 1377. 1330.    0. 1560. 1381.]
 [1460. 1439. 2322. 1454. 1488. 1518. 1548. 1560.    0. 1551.]
 [1353. 1432. 2240. 1340. 1329. 1357. 1334. 1381. 1551.    0.]]

Execution time for delta_score: 0.0000 seconds


### KS

In [15]:
def ks(models, x_test):
    num_samples = x_test.shape[0]
    num_models = len(models)

    # مرحله 1: پیش‌بینی کلاس‌ها برای داده‌های تست توسط هر مدل
    argmax_preds = np.array([model.predict(x_test).argmax(axis=1) for model in client_models])

    # مرحله 2: یافتن کلاس پرتکرار برای هر نمونه
    most_frequent_classes = [Counter(argmax_preds[:, i]).most_common(1)[0][0] for i in range(num_samples)]

    # مرحله 3: استخراج احتمال کلاس پرتکرار از تمامی مدل‌ها برای هر نمونه
    all_probabilities = np.zeros((num_samples, num_clients))
    for i, model in enumerate(client_models):
        predictions = model.predict(x_test)  # احتمالات کلاس‌های مختلف برای هر نمونه
        for j, sample in enumerate(x_test):
            most_frequent_class = most_frequent_classes[j]
            all_probabilities[j, i] = predictions[j, most_frequent_class]  # احتمال کلاس پرتکرار
    # مرحله 4: انجام تست KS بین مدل‌های کلاینت و ایجاد ماتریس KS
    ks_distance_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            # تست KS برای مقایسه احتمالات دو مدل کلاینت
            ks_statistic, _ = ks_2samp(all_probabilities[:, i], all_probabilities[:, j])
            ks_distance_matrix[i, j] = ks_statistic
            ks_distance_matrix[j, i] = ks_statistic  # ماتریس متقارن

    return ks_distance_matrix

# اجرای تابع با تعیین تعداد ارقام اعشار
matrix_ks = ks(client_models, x_test)
print("Distance Matrix for KS:")
print(matrix_ks)

start_time = time.time()
end_time = time.time()
execution_time = end_time - start_time
print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Distance Matrix for KS:
[[0.     0.0636 0.8096 0.0335 0.0474 0.0822 0.0569 0.0338 0.0154 0.056 ]
 [0.0636 0.     0.7943 0.0484 0.0298 0.0307 0.0298 0.0578 0.0702 0.0159]
 [0.8096 0.7943 0.     0.7993 0.8006 0.79   0.7977 0.799  0.8048 0.8083]
 [0.0335 0.0484 0.7993 0.     0.0218 0.0677 0.0263 0.013  0.0421 0.0401]
 [0.0474 0.0298 0.8006 0.0218 0.     0.0492 0.0133 0.0309 0.0562 0.0223]
 [0.0822 0.0307 0.79   0.0677 0.0492 0.     0.0477 0.0764 0.0887 0.0316]
 [0.0569 0.0298 0.7977 0.0263 0.0133 0.0477 0.     0.0348 0.0662 0.0235]
 [0.0338 0.0578 0.799  0.013  0.0309 0.0764 0.0348 0.     0.0425 0.0502]
 [0.0154 0.0702 0.8048 0.0421 0.0562 0.0887 0.0662 0.0425 0.     0.0646]
 [0.056  0.0159 0.8083 0.0401 0.0223 0.0316 0.0235 0.0502 0.0646 0.    ]]

Execution time for delta_class: 0.0000 seconds


### Chi2

In [20]:
def chi_square(models, x_test):
    num_models = len(models)
    num_samples = x_test.shape[0]
    n_classes = 10  # تعداد کلاس ها

    # ساخت ماتریس متقارن برای نتایج Chi-Square
    chi2_matrix = np.zeros((num_models, num_models))
    all_probabilities = np.zeros((num_samples, num_models))

    for i, model in enumerate(models):
        predictions = model.predict(x_test)  # احتمالات کلاس‌های مختلف برای هر نمونه
        for j, sample in enumerate(x_test):
            all_probabilities[j, i] = predictions[j].argmax()

    # مرحله: ایجاد جدول‌های متقاطع و انجام تست Chi-Square
    for i in range(num_models):
        for j in range(i + 1, num_models):
            # ساخت جدول متقاطع
            contingency_table = pd.crosstab(all_probabilities[:, i], all_probabilities[:, j])
            chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
            chi2_matrix[i, j] = chi2_stat
            chi2_matrix[j, i] = chi2_stat  # ماتریس متقارن

    return chi2_matrix

# اجرای تابع با تعیین تعداد ارقام اعشار
matrix_chi2 = chi_square(client_models, x_test)
print("Distance Matrix for Chi square:")
print(matrix_chi2)

start_time = time.time()
end_time = time.time()
execution_time = end_time - start_time
print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Distance Matrix for Chi square:
[[    0.         79333.01195652 62103.99227715 79755.88003874
  79818.98928157 79774.76958452 80079.77541876 79888.34058736
  78766.21570532 80191.84790918]
 [79333.01195652     0.         62473.95067528 79515.33467231
  79576.35733202 78886.45720699 78708.75179731 79053.9194527
  79380.90792853 78996.16413276]
 [62103.99227715 62473.95067528     0.         63043.04652187
  62921.7477454  62530.39733589 62316.1127854  63625.78250606
  61903.58473286 62438.85796303]
 [79755.88003874 79515.33467231 63043.04652187     0.
  79791.01971917 80234.08191857 80391.9478384  79865.16029808
  78897.47065496 80244.46360743]
 [79818.98928157 79576.35733202 62921.7477454  79791.01971917
      0.         79643.9748238  79104.76777233 80265.22401841
  78866.81466973 80229.86145314]
 [79774.76958452 78886.45720699 62530.39733589 80234.08191857
  79643.9748238      0.         79753.08197036 79882.84787126
  77831.36021879 80268.94856771]
 [80079.77541876 78708.75179731 623

### The problematic client

In [25]:
def meancal(matrix):
    temp = 0
    x = matrix.shape[0]    
    arrmean = []
    
    for i in range(0,x):
        #print(matrix[i].mean())
        temp = ((matrix[i].mean())*10)/9
        arrmean.append(temp)
    return arrmean

def iqrfunc(nparray, multiplier = 1.5):
    data = np.array(nparray)
    q1 = np.percentile(data,25)
    q3 = np.percentile(data,75)
    iqr = q3 -q1
    lower_bound = q1-(multiplier*1.5)
    upper_bound = q3+(multiplier*1.5)
    outliers = np.where((data<lower_bound) | (data>upper_bound))[0]
    return outliers
    
diff_class = delta_class(client_models, x_test)
diff_score = delta_score(client_models, x_test,1)
diff_ks = ks(client_models, x_test)
diff_chi2 = chi_square(client_models, x_test)

temp_c = meancal(diff_class)
temp_s = meancal(diff_score)
temp_ks = meancal(diff_ks)
temp_chi2 = meancal(diff_chi2)

print(f"Problemmatic Clients for Delta class {iqrfunc(temp_c)}:")                       
print(f"Problemmatic Clients for Delta Score {iqrfunc(temp_s)}:")                                            
print(f"Problemmatic Clients for KS {iqrfunc(temp_ks)}:")                       
print(f"Problemmatic Clients for X2 {iqrfunc(temp_chi2)}:") 

Problemmatic Clients for Delta class [2 3 4 8]:
Problemmatic Clients for Delta Score [2 3 4 8]:
Problemmatic Clients for KS []:
Problemmatic Clients for X2 [1 2 3 4 8 9]:


In [31]:
import numpy as np

# فرض کنید این لیست کلاینت‌های باگ‌دار واقعی است
# ابتدا از خروجی تحلیل‌ها برای شناسایی کلاینت‌های باگ‌دار استفاده می‌کنیم
true_problematic_clients = []

# برای هر نوع تحلیل
for title, diff in zip(["Delta class", "Delta Score", "KS", "X2"], 
                       [temp_c, temp_s, temp_ks, temp_chi2]):
    
    # تبدیل لیست به آرایه NumPy
    diff_array = np.array(diff)  
    
    # ایندکس کلاینت‌های باگ‌دار
    problematic_clients = iqrfunc(meancal(diff_array))
    print(f"Problemmatic Clients for {title} {problematic_clients}:")
    
    # به‌روزرسانی لیست کلاینت‌های باگ‌دار واقعی
    true_problematic_clients.extend(problematic_clients.tolist())

# تبدیل به آرایه numpy و حذف تکراری‌ها
true_problematic_clients = np.unique(np.array(true_problematic_clients))

# توابع برای محاسبه معیارها
def precision(predicted, actual):
    tp = len(np.intersect1d(predicted, actual))  # True Positives
    fp = len(np.setdiff1d(predicted, actual))     # False Positives
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def recall(predicted, actual):
    tp = len(np.intersect1d(predicted, actual))  # True Positives
    fn = len(np.setdiff1d(actual, predicted))     # False Negatives
    return tp / (tp + fn) if (tp + fn) > 0 else 0

def f1_score(predicted, actual):
    p = precision(predicted, actual)
    r = recall(predicted, actual)
    return 2 * (p * r) / (p + r) if (p + r) > 0 else 0

def accuracy(predicted, actual, total_clients):
    tp = len(np.intersect1d(predicted, actual))  # True Positives
    tn = total_clients - len(actual) - len(predicted) + len(np.intersect1d(predicted, actual))  # True Negatives
    return (tp + tn) / total_clients

# محاسبه معیارها برای هر خروجی
for title, diff in zip(["Delta class", "Delta Score", "KS", "X2"], 
                       [temp_c, temp_s, temp_ks, temp_chi2]):
    
    # تبدیل لیست به آرایه NumPy
    diff_array = np.array(diff)
    
    # ایندکس کلاینت‌های باگ‌دار
    problematic_clients = iqrfunc(meancal(diff_array))
    print(f"\nProblemmatic Clients for {title} {problematic_clients}:")
    
    # محاسبه معیارها
    p = precision(problematic_clients, true_problematic_clients)
    r = recall(problematic_clients, true_problematic_clients)
    f1 = f1_score(problematic_clients, true_problematic_clients)
    acc = accuracy(problematic_clients, true_problematic_clients, len(client_models))

    # نمایش نتایج
    print(f"Precision: {p:.2f}, Recall: {r:.2f}, F1 Score: {f1:.2f}, Accuracy: {acc:.2f}\n")


Problemmatic Clients for Delta class [2 3 4 8]:
Problemmatic Clients for Delta Score [2 3 4 8]:
Problemmatic Clients for KS []:
Problemmatic Clients for X2 [1 2 3 4 8 9]:

Problemmatic Clients for Delta class [2 3 4 8]:
Precision: 1.00, Recall: 0.67, F1 Score: 0.80, Accuracy: 0.80


Problemmatic Clients for Delta Score [2 3 4 8]:
Precision: 1.00, Recall: 0.67, F1 Score: 0.80, Accuracy: 0.80


Problemmatic Clients for KS []:
Precision: 0.00, Recall: 0.00, F1 Score: 0.00, Accuracy: 0.40


Problemmatic Clients for X2 [1 2 3 4 8 9]:
Precision: 1.00, Recall: 1.00, F1 Score: 1.00, Accuracy: 1.00

